# Creating the Dataset for 2017

This Notebook: 
 - Reads in our All Public Schools data for 2017
 - Creates a new column for low performing schools and recurring low performing schools 
 - Cleans up the dataset for ML capabilities

**The Final Datasets**: 
 - Includes whether school is low performing in 2014, 2015, 2016, and 2017 represented as: 
 - LPS_14, LPS_15, LPS_16, LPS_17, and RLPS which is a categorical variable between 0 and 4 representing how many times in the past 4 school years a school has been low performing. 
 - Removes Charter Schools
 
**Datasets ending in LPS_Processed are preprocessed for Machine Learning and go through the following transformations: **
1. Missing student body racial compositions are imputed using district averages.
2. Columns that have the same value in every single row are deleted.
3. Columns that have a unique value in every single row (all values are different) are deleted.
4. Empty columns (all values are NA or NULL) are deleted.
5. Numeric columns with more than the percentage of missing values specified by the *missingThreshold* parameter.
6. Remaining numeric, non-race columns with missing values are imputed / populated with 0.  In many cases, schools are not reporting values when they are zero. However, mean imputation or some other more sophisticated strategy might be considered here.
7. Categorical / text based columns with > *uniqueThreshold* unique values are deleted.
8. All remaining categorical / text based columns are one-hot encoded.  In categorical columns, one-hot encoding creates one new boolean / binary field per unique value in the target column, converting all categorical columns to a numeric data type. 


*Code Adapted from: Dr. Jacob Drew. Code can be found: https://github.com/jakemdrew/EducationDataNC*

In [1]:
#import required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import os

import seaborn as sns
sns.set(color_codes=True)

import warnings
warnings.filterwarnings("ignore")

In [2]:
outputDir = os.getcwd() + "/"
cwd = os.getcwd()
os.chdir("..")
cwd = os.getcwd()
cwd = cwd + "/"

Read in full datasets from school years 2013-2014, 2014-2015, 2015-2016, 2016-2017.

In [3]:
#csv path
schoolYear = 2017

publicSchools14 = pd.read_csv(cwd + "2014/PublicSchools2014.csv", low_memory = False)
print(publicSchools14.info())

publicSchools15 = pd.read_csv(cwd + "2015/PublicSchools2015.csv", low_memory = False)
print(publicSchools15.info())

publicSchools16 = pd.read_csv(cwd + "2016/PublicSchools2016.csv", low_memory = False)
print(publicSchools16.info())

publicSchools17 = pd.read_csv(outputDir + "PublicSchools2017.csv", low_memory = False)
print(publicSchools17.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2561 entries, 0 to 2560
Columns: 365 entries, vphone_ad to WhitePct
dtypes: float64(331), int64(3), object(31)
memory usage: 7.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2585 entries, 0 to 2584
Columns: 362 entries, vphone_ad to WhitePct
dtypes: float64(315), int64(3), object(44)
memory usage: 7.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2599 entries, 0 to 2598
Columns: 362 entries, vphone_ad to WhitePct
dtypes: float64(318), int64(3), object(41)
memory usage: 7.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2617 entries, 0 to 2616
Columns: 385 entries, vphone_ad to Number_Industry_Recognized_Crede
dtypes: float64(340), int64(3), object(42)
memory usage: 7.7+ MB
None


### Low Performing School Definition:

“Low-performing schools are those that receive a school performance grade of D or F and a school growth score of "met expected growth" or "not met expected growth" as defined by G.S. 115C-83.15.” (G.S. 115C-105.37(a)), and

http://www.dpi.state.nc.us/schooltransformation/low-performing/

In [4]:
#create new column that equals 1 if low Performing
publicSchools14['LPS'] = np.where(publicSchools14['SPG Grade'].isin(['F', 'D'])&
                            publicSchools14['EVAAS Growth Status'].isin(['NotMet', 'Met']),
                            1, 0)

publicSchools15['LPS'] = np.where(publicSchools15['SPG Grade'].isin(['F', 'D'])&
                            publicSchools15['EVAAS Growth Status'].isin(['NotMet', 'Met']),
                            1, 0)

publicSchools16['LPS'] = np.where(publicSchools16['SPG Grade'].isin(['F', 'D'])&
                            publicSchools16['EVAAS Growth Status'].isin(['NotMet', 'Met']),
                            1, 0)

publicSchools17['LPS'] = np.where(publicSchools17['SPG Grade'].isin(['F', 'D'])&
                            publicSchools17['EVAAS Growth Status'].isin(['NotMet', 'Met']),
                            1, 0)


In [5]:
## Get the unit_codes of the low performing schools for each year and create new columns in the 2017 dataset
## to reflect the years of low performance
unit_codes_14 = publicSchools14['unit_code'][publicSchools14.LPS == 1]
unit_codes_15 = publicSchools15['unit_code'][publicSchools15.LPS == 1]
unit_codes_16 = publicSchools16['unit_code'][publicSchools16.LPS == 1]
unit_codes_17 = publicSchools17['unit_code'][publicSchools17.LPS == 1]

In [6]:
##Creat columns in publicSchools17 dataset
publicSchools17['LPS_14'] = np.where(publicSchools17['unit_code'].isin(unit_codes_14), 1, 0)
print('************** 2014 **************')
print('Num Public Schools Low Performing 2013-14: ', len(unit_codes_14))
print('Num Schools Still Running in 2017: ', len(publicSchools17.LPS_14[publicSchools17.LPS_14 ==1]))
print('Difference: ', (len(unit_codes_14) - len(publicSchools17.LPS_14[publicSchools17.LPS_14 ==1])))

publicSchools17['LPS_15'] = np.where(publicSchools17['unit_code'].isin(unit_codes_15), 1, 0)
print('************** 2015 **************')
print('Num Public Schools Low Performing 2014-15: ', len(unit_codes_15))
print('Num Schools Still Running in 2017: ', len(publicSchools17.LPS_15[publicSchools17.LPS_15 ==1]))
print('Difference: ', (len(unit_codes_15) - len(publicSchools17.LPS_15[publicSchools17.LPS_15 ==1])))

publicSchools17['LPS_16'] = np.where(publicSchools17['unit_code'].isin(unit_codes_16), 1, 0)
print('************** 2016 **************')
print('Num Public Schools Low Performing 2015-16: ', len(unit_codes_16))
print('Num Schools Still Running in 2017: ', len(publicSchools17.LPS_16[publicSchools17.LPS_16 ==1]))
print('Difference: ', (len(unit_codes_16) - len(publicSchools17.LPS_16[publicSchools17.LPS_16 ==1])))

publicSchools17['LPS_17'] = np.where(publicSchools17['unit_code'].isin(unit_codes_17), 1, 0)
print('************** 2014 **************')
print('Num Public Schools Low Performing 2016-17: ', len(unit_codes_17))
print('Num Schools Still Running in 2017: ', len(publicSchools17.LPS_17[publicSchools17.LPS_17 ==1]))
print('Difference: ', (len(unit_codes_17) - len(publicSchools17.LPS_17[publicSchools17.LPS_17 ==1])))

************** 2014 **************
Num Public Schools Low Performing 2013-14:  615
Num Schools Still Running in 2017:  600
Difference:  15
************** 2015 **************
Num Public Schools Low Performing 2014-15:  580
Num Schools Still Running in 2017:  572
Difference:  8
************** 2016 **************
Num Public Schools Low Performing 2015-16:  488
Num Schools Still Running in 2017:  485
Difference:  3
************** 2014 **************
Num Public Schools Low Performing 2016-17:  505
Num Schools Still Running in 2017:  505
Difference:  0


In [7]:
publicSchools17 = publicSchools17.drop(columns = 'LPS', axis=1)

## Number of times the school has been low performing

In [8]:
## Times low performing 
RLPS_series = []
for index, row in publicSchools17.iterrows():
    if row['LPS_14'] + row['LPS_15'] + row['LPS_16'] + row['LPS_17']  == 1:
        RLPS_series.append(1)
    elif row['LPS_14'] + row['LPS_15'] + row['LPS_16'] + row['LPS_17']  == 2:
        RLPS_series.append(2)
    elif row['LPS_14'] + row['LPS_15'] + row['LPS_16'] + row['LPS_17']  == 3:
        RLPS_series.append(3)
    elif row['LPS_14'] + row['LPS_15'] + row['LPS_16'] + row['LPS_17']  == 4:
        RLPS_series.append(4)
    else: 
        RLPS_series.append(0)
print(np.unique(RLPS_series))
publicSchools17['RLPS'] = RLPS_series

[0 1 2 3 4]


In [9]:
## Check Work: 
school_LPS_1 = publicSchools17.unit_code[publicSchools17['RLPS'] == 1]
school_LPS_2 = publicSchools17.unit_code[publicSchools17['RLPS'] == 2]
school_LPS_3 = publicSchools17.unit_code[publicSchools17['RLPS'] == 3]
school_LPS_4 = publicSchools17.unit_code[publicSchools17['RLPS'] == 4]
school_LPS_0 = publicSchools17.unit_code[publicSchools17['RLPS'] == 0]


print('Number of Schools Low Performing One Year: ', (school_LPS_1.count()))
print('Number of Schools Low Performing Two Years: ', (school_LPS_2.count()))
print('Number of Schools Low Performing Three Years: ', (school_LPS_3.count()))
print('Number of Schools Low Performing Four Years: ', (school_LPS_4.count()))
print('Number of Schools Low Performing Never: ', (school_LPS_0.count()))

Number of Schools Low Performing One Year:  263
Number of Schools Low Performing Two Years:  227
Number of Schools Low Performing Three Years:  203
Number of Schools Low Performing Four Years:  209
Number of Schools Low Performing Never:  1715


# Create Machine Learning Dataset for 2014

In [10]:
schoolData = publicSchools17.copy()

## Define Missing Data Thresholds
- Missing data threshold is .20% 
- Unique Categorical Data Threshold is 25

In [11]:
#Missing Data Threshold (Per Column)
missingThreshold = 0.20

#Unique Value Threshold (Per Column)
#Delete Columns >  uniqueThreshold unique values prior to one-hot encoding. 
#(each unique value becomes a new column during one-hot encoding)
uniqueThreshold = 25

print('*********Start: Beginning Column and Row Counts********************************************')
schoolData.info(verbose=False)

#Select only public schools as charter schools are missing data for many columns.
schoolData = schoolData[(schoolData['type_cd'] == 'P') & (schoolData['student_num'] > 0)]

print('\r\n*********After: Selecting Only Public School Campuses**********************************')
schoolData.info(verbose=False)

#Save primary key
unit_code = schoolData['unit_code']
#Convert zip code to string
schoolData['szip_ad'] = schoolData['szip_ad'].astype('object')

*********Start: Beginning Column and Row Counts********************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2617 entries, 0 to 2616
Columns: 390 entries, vphone_ad to RLPS
dtypes: float64(340), int64(8), object(42)
memory usage: 7.8+ MB

*********After: Selecting Only Public School Campuses**********************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2443 entries, 2 to 2616
Columns: 390 entries, vphone_ad to RLPS
dtypes: float64(340), int64(8), object(42)
memory usage: 7.3+ MB


## Student Body Racial Composition Features 
**Impute / update missing Student Body Racial Composition Fields using mean imputation.**
* When there are no racial composition percentages for a particular school campus / unit_code, fill in the missing values 

In [12]:
# #Get Student Body Racial Composition Fields
# raceCompositionFields = schoolData.filter(regex='Indian|Asian|Hispanic|Black|White|PacificIsland|TwoOrMore|Minority')\
#                                   .filter(regex='Pct').columns
    
# rowsBefore = schoolData[raceCompositionFields].isnull().T.any().T.sum()

# #Update missing race values with the district average when avaiable (No district averages for charter schools) 
# schoolData[raceCompositionFields] = schoolData.groupby('District Name')[raceCompositionFields]\
#                                               .transform(lambda x: x.fillna(x.mean()))

#     #Review dataset contents after Racial Composition Imputation
# print('*********After: Updating Missing Racial Compostion Values****************************')   
# rowsAfter = schoolData[raceCompositionFields].isnull().T.any().T.sum()
# rowsUpdated = rowsBefore - rowsAfter
# print ('Rows Updated / Imputed: ', rowsUpdated) 
# print('\r\nTotal Rows Missing Racial Compositions By District Name') 
# schoolData['District Name'][schoolData[raceCompositionFields].isnull().T.any().T].value_counts()
# print(schoolData['District Name'][schoolData[raceCompositionFields].isnull().T.any().T].value_counts())

## Remove Columns with Problematic Data
**Here we remove entire columns that could cause problems during machine learning.  The following operations are performed:**
* Remove any columns that have the same value in every single row.
* Remove any columns that have a unique value in every single row (all values are different).
* Remove empty columns (all values are NA or NULL).### Remove Columns with the same data in all rows

### Same Value in Every Single Row

In [13]:
#Remove any fields that have the same value in all rows
UniqueValueCounts = schoolData.nunique(dropna=False)
SingleValueCols = UniqueValueCounts[UniqueValueCounts == 1].index
schoolData = schoolData.drop(SingleValueCols, axis=1)

#Review dataset contents after drops
print('*********After: Removing fields with the same value in every row.*******************')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(SingleValueCols))
print('Columns:', list(SingleValueCols))

*********After: Removing fields with the same value in every row.*******************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2443 entries, 2 to 2616
Columns: 265 entries, vphone_ad to RLPS
dtypes: float64(221), int64(5), object(39)
memory usage: 5.0+ MB

Columns Deleted:  125
Columns: ['year', 'state_ad', 'type_cd', 'closed_ind', 'new_ind', 'super_nm', 'State_Name', 'stem', 'total_expense_num', 'salary_expense_pct', 'benefits_expense_pct', 'services_expense_pct', 'supplies_expense_pct', 'instruct_equip_exp_pct', 'other_expense_pct', 'federal_perpupil_num', 'local_perpupil_num', 'state_perpupil_num', 'lea_benefits_expense_pct', 'lea_other_expense_pct', 'st_total_expense_num', 'st_salary_expense_pct', 'st_benefits_expense_pct', 'st_services_expense_pct', 'st_supplies_expense_pct', 'st_instruct_equip_exp_pct', 'st_other_expense_pct', 'st_federal_perpupil_num', 'st_local_perpupil_num', 'st_state_perpupil_num', 'building_expense_pct', 'lea_building_expense_pct', 'st_building_expen

### Remove Columns with Unique Data In All Rows

In [14]:
#Remove any fields that have unique values in every row
schoolDataRecordCt = schoolData.shape[0]
UniqueValueCounts = schoolData.apply(pd.Series.nunique)
AllUniqueValueCols = UniqueValueCounts[UniqueValueCounts == schoolDataRecordCt].index
schoolData = schoolData.drop(AllUniqueValueCols, axis=1)

#Review dataset contents after drops
print('*********After: Removing columns with unique values in every row.*******************')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(AllUniqueValueCols))

*********After: Removing columns with unique values in every row.*******************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2443 entries, 2 to 2616
Columns: 263 entries, vphone_ad to RLPS
dtypes: float64(221), int64(5), object(37)
memory usage: 4.9+ MB

Columns Deleted:  2


### Remove null/blank values in every row 

In [15]:
#Remove any empty fields (null values in every row)
schoolDataRecordCt = schoolData.shape[0]
NullValueCounts = schoolData.isnull().sum()
NullValueCols = NullValueCounts[NullValueCounts == schoolDataRecordCt].index
schoolData = schoolData.drop(NullValueCols, axis=1)

#Review dataset contents after empty field drops
print('*********After: Removing columns with null / blank values in every row.*************')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(NullValueCols))

*********After: Removing columns with null / blank values in every row.*************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2443 entries, 2 to 2616
Columns: 263 entries, vphone_ad to RLPS
dtypes: float64(221), int64(5), object(37)
memory usage: 4.9+ MB

Columns Deleted:  0


### Data Types

In [16]:
#Isolate continuous and categorical data types
#These are indexers into the schoolData dataframe and may be used similar to the schoolData dataframe 
sD_boolean = schoolData.loc[:, (schoolData.dtypes == bool) ]
sD_nominal = schoolData.loc[:, (schoolData.dtypes == object)]
sD_continuous = schoolData.loc[:, (schoolData.dtypes != bool) & (schoolData.dtypes != object)]
print ("Boolean Columns: ", sD_boolean.shape[1])
print ("Nominal Columns: ", sD_nominal.shape[1])
print ("Continuous Columns: ", sD_continuous.shape[1])
print ("Columns Accounted for: ", sD_nominal.shape[1] + sD_continuous.shape[1] + sD_boolean.shape[1])

Boolean Columns:  0
Nominal Columns:  37
Continuous Columns:  226
Columns Accounted for:  263


### Convert Booleans to 1s and 0s

In [17]:
#Convert Columns with values of nan, 'Y' as booleans
hasY = []
for x in schoolData.columns: 
    if ((['Y'] in schoolData[x].unique()) & (len(schoolData[x].unique()) == 2)):
        hasY.append(x)
print(hasY)

for x in hasY: 
# Map flag fields into bool
    schoolData[x] = schoolData[x].map({'Y':1, np.nan:0})

schoolData['esea_status'] = schoolData['esea_status'].map({'P':'Esea_Pass', 'F':'Esea_Fail', np.nan:'Non_Esea'})
schoolData['Grad_project_status'] = schoolData['Grad_project_status'].map({'Y':1, 'N':0, np.nan:0})

#Boolean Columns
sD_boolean = schoolData.loc[:, (schoolData.dtypes == bool) ]
print ("Boolean Columns: ", sD_boolean.shape[1])

['title1_type_cd', 'clp_ind', 'focus_clp_ind', 'summer_program_ind', 'asm_no_spg_ind', 'no_data_spg_ind']
Boolean Columns:  0


### Scale Expression of Percentages

In [18]:
PercentageFields = schoolData.filter(regex='pct|Pct|percent|Percent').columns
converted = []
for x in PercentageFields:
    if(schoolData[x].max() > 1):
        converted.append(x)
        schoolData[x] = schoolData[x]/100
print('Number of Percentage Fields:', len(PercentageFields))
print(list(PercentageFields))
print('Percentage Fields Converted to between 0 and 1: ', len(converted))

Number of Percentage Fields: 124
['lea_salary_expense_pct', 'lea_services_expense_pct', 'lea_supplies_expense_pct', 'lea_instruct_equip_exp_pct', 'ACT_pTarget_PctMet', 'ACTWorkKeys_pTarget_PctMet', 'CurrentYearEOC_pTarget_PctMet', 'MathGr10_pTarget_PctMet', 'MathGr3-8_pTarget_PctMet', 'ReadingGr10_pTarget_PctMet', 'ReadingGr3-8_pTarget_PctMet', 'SciGr11_pTarget_PctMet', 'SciGr5&8_pTarget_PctMet', 'TotalTargets_pTarget_PctMet', 'sat_participation_pct', 'lea_sat_participation_pct', 'ap_participation_pct', 'lea_ap_participation_pct', 'ap_pct_3_or_above', 'lea_ap_pct_3_or_above', 'ib_participation_pct', 'ib_pct_4_or_above', 'lea_ib_pct_4_or_above', 'ALL_All Students (Total or Subtotal_ENROLL_sch_pct', 'ECODIS_Economically Disadvantaged_ENROLL_sch_pct', 'F_Female_ENROLL_sch_pct', 'LEP_Limited English Proficiency_ENROLL_sch_pct', 'M_Male_ENROLL_sch_pct', 'MA_Asian_ENROLL_sch_pct', 'MAN_American Indian_ENROLL_sch_pct', 'MB_Black_ENROLL_sch_pct', 'MHL_Hispanic_ENROLL_sch_pct', 'MM_Multiracial_

### Removal of STATE variables

In [19]:
stateFields = schoolData.filter(regex='st_').columns
print (stateFields)

keepFields = [col for col in schoolData.columns if col not in stateFields]
schoolData = schoolData[keepFields]
print(schoolData.info())

Index(['st_avg_student_num', 'st_total_specialized_courses',
       'st_ap_ib_courses', 'st_cte_courses', 'st_univ_college_courses',
       'st_avg_daily_attend_pct', 'st_crime_per_c_num',
       'st_short_susp_per_c_num', 'st_long_susp_per_c_num',
       'st_expelled_per_c_num', 'st_flicensed_teach_pct',
       'st_tchyrs_0thru3_pct', 'st_tchyrs_4thru10_pct', 'st_tchyrs_11plus_pct',
       'st_class_teach_num', 'st_nbpts_num', 'st_advance_dgr_pct',
       'st_1yr_tchr_trnovr_pct', 'st_emer_prov_teach_pct',
       'st_lateral_teach_pct'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2443 entries, 2 to 2616
Columns: 243 entries, vphone_ad to RLPS
dtypes: float64(201), int64(12), object(30)
memory usage: 4.5+ MB
None


## Subset Data by Category before any more PreProcessing
By subsetting the dataset early, we avoid deleting columns that do not meet our missing data threshold because they may only be valid for certain schools (ie, graduate rate only collected for High Schools)

In [20]:
print('*********************************All Public Schools****************************')
schoolData.info(verbose=False)

#Filter regular public high schools
HighSchools = schoolData[((schoolData.category_cd == 'H') | 
                             (schoolData.category_cd == 'T') | 
                             (schoolData.category_cd == 'A')) &
                             (schoolData.student_num > 0) & 
                             (schoolData.school_type_txt == 'Regular School')
                            ]

print('*********************************Regular Public High Schools*******************')
HighSchools.info(verbose=False)

#Filter regular public middle schools
MiddleSchools = schoolData[((schoolData.category_cd == 'M') | 
                               (schoolData.category_cd == 'T') | 
                               (schoolData.category_cd == 'A') |
                               (schoolData.category_cd == 'I')) &
                               (schoolData.student_num > 0) & 
                               (schoolData.school_type_txt == 'Regular School')
                             ]

print('*********************************Regular Public Middle Schools******************')
MiddleSchools.info(verbose=False)


#Filter regular elementary high schools
ElementarySchools = schoolData[((schoolData.category_cd == 'E') | 
                                   (schoolData.category_cd == 'I') | 
                                   (schoolData.category_cd == 'A')) &
                                   (schoolData.student_num > 0) & 
                                   (schoolData.school_type_txt == 'Regular School')
                                 ]

print('*********************************Regular Public Elementary Schools**************')
ElementarySchools.info(verbose=False)

*********************************All Public Schools****************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2443 entries, 2 to 2616
Columns: 243 entries, vphone_ad to RLPS
dtypes: float64(201), int64(12), object(30)
memory usage: 4.5+ MB
*********************************Regular Public High Schools*******************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 470 entries, 2 to 2615
Columns: 243 entries, vphone_ad to RLPS
dtypes: float64(201), int64(12), object(30)
memory usage: 895.9+ KB
*********************************Regular Public Middle Schools******************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 526 entries, 5 to 2615
Columns: 243 entries, vphone_ad to RLPS
dtypes: float64(201), int64(12), object(30)
memory usage: 1002.7+ KB
*********************************Regular Public Elementary Schools**************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1268 entries, 3 to 2616
Columns: 243 entries, vphone_ad to RLPS
dtypes: float64(201),

## Isolate Datatypes for each subset

In [21]:
#Isolate continuous and categorical data types
#These are indexers into the schoolData dataframe and may be used similar to the schoolData dataframe 
sD_boolean = schoolData.loc[:, (schoolData.dtypes == bool) ]
sD_nominal= schoolData.loc[:, (schoolData.dtypes == object)]
sD_continuous = schoolData.loc[:, (schoolData.dtypes != bool) & (schoolData.dtypes != object)]
print("*********** ALL SCHOOLS **********")
print ("Boolean Columns: ", sD_boolean.shape[1])
print ("Nominal Columns: ", sD_nominal.shape[1])
print ("Continuous Columns: ", sD_continuous.shape[1])
print ("Columns Accounted for: ", sD_nominal.shape[1] + sD_continuous.shape[1] + sD_boolean.shape[1])

#Isolate continuous and categorical data types
#These are indexers into the schoolData dataframe and may be used similar to the schoolData dataframe 
sD_boolean_High = HighSchools.loc[:, (HighSchools.dtypes == bool) ]
sD_nominal_High = HighSchools.loc[:, (HighSchools.dtypes == object)]
sD_continuous_High = HighSchools.loc[:, (HighSchools.dtypes != bool) & (HighSchools.dtypes != object)]
print("*********** High SCHOOLS **********")
print ("Boolean Columns: ", sD_boolean_High.shape[1])
print ("Nominal Columns: ", sD_nominal_High.shape[1])
print ("Continuous Columns: ", sD_continuous_High.shape[1])
print ("Columns Accounted for: ", sD_nominal_High.shape[1] + sD_continuous_High.shape[1] + sD_boolean_High.shape[1])

#Isolate continuous and categorical data types
#These are indexers into the schoolData dataframe and may be used similar to the schoolData dataframe 
sD_boolean_Middle = MiddleSchools.loc[:, (MiddleSchools.dtypes == bool) ]
sD_nominal_Middle = MiddleSchools.loc[:, (MiddleSchools.dtypes == object)]
sD_continuous_Middle = HighSchools.loc[:, (MiddleSchools.dtypes != bool) & (MiddleSchools.dtypes != object)]
print("*********** Middle SCHOOLS **********")
print ("Boolean Columns: ", sD_boolean_Middle.shape[1])
print ("Nominal Columns: ", sD_nominal_Middle.shape[1])
print ("Continuous Columns: ", sD_continuous_Middle.shape[1])
print ("Columns Accounted for: ", sD_nominal_Middle.shape[1] + sD_continuous_Middle.shape[1] + sD_boolean_Middle.shape[1])

#Isolate continuous and categorical data types
#These are indexers into the schoolData dataframe and may be used similar to the schoolData dataframe 
sD_boolean_Elementary = ElementarySchools.loc[:, (ElementarySchools.dtypes == bool) ]
sD_nominal_Elementary = ElementarySchools.loc[:, (ElementarySchools.dtypes == object)]
sD_continuous_Elementary = ElementarySchools.loc[:, (ElementarySchools.dtypes != bool) & (ElementarySchools.dtypes != object)]
print("*********** Elementary SCHOOLS **********")
print ("Boolean Columns: ", sD_boolean_Elementary.shape[1])
print ("Nominal Columns: ", sD_nominal_Elementary.shape[1])
print ("Continuous Columns: ", sD_continuous_Elementary.shape[1])
print ("Columns Accounted for: ", sD_nominal_Elementary.shape[1] + sD_continuous_Elementary.shape[1] + sD_boolean_Elementary.shape[1])

*********** ALL SCHOOLS **********
Boolean Columns:  0
Nominal Columns:  30
Continuous Columns:  213
Columns Accounted for:  243
*********** High SCHOOLS **********
Boolean Columns:  0
Nominal Columns:  30
Continuous Columns:  213
Columns Accounted for:  243
*********** Middle SCHOOLS **********
Boolean Columns:  0
Nominal Columns:  30
Continuous Columns:  213
Columns Accounted for:  243
*********** Elementary SCHOOLS **********
Boolean Columns:  0
Nominal Columns:  30
Continuous Columns:  213
Columns Accounted for:  243


### Eliminate columns with > 20% Data Missing

In [22]:
#Eliminate continuous columns with more than missingThreshold percentage of missing values
schoolDataRecordCt = sD_continuous.shape[0]
missingValueLimit = schoolDataRecordCt * missingThreshold
NullValueCounts = sD_continuous.isnull().sum()
NullValueCols = NullValueCounts[NullValueCounts >= missingValueLimit].index
schoolData = schoolData.drop(NullValueCols, axis=1)

#Review dataset contents after empty field drops
print('\n')
print('*********After: Removing columns with >= missingThreshold % of missing values******')
print("************* All Schools *************")
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(NullValueCols))
print ('\r\nColumns: ', list(NullValueCols))

#Eliminate continuous columns with more than missingThreshold percentage of missing values
schoolDataRecordCt_High = sD_continuous_High.shape[0]
missingValueLimit_High = schoolDataRecordCt_High * missingThreshold
NullValueCounts_High = sD_continuous_High.isnull().sum()
NullValueCols_High = NullValueCounts_High[NullValueCounts_High >= missingValueLimit].index
HighSchools = HighSchools.drop(NullValueCols_High, axis=1)

#Review dataset contents after empty field drops
print('\n')
print('*********After: Removing columns with >= missingThreshold % of missing values******')
print("************* High Schools *************")
HighSchools.info(verbose=False)
print ('\r\nColumns Deleted: ', len(NullValueCols_High))
print ('\r\nColumns: ', list(NullValueCols_High))

#Eliminate continuous columns with more than missingThreshold percentage of missing values
schoolDataRecordCt_Middle = sD_continuous_Middle.shape[0]
missingValueLimit_Middle = schoolDataRecordCt_Middle * missingThreshold
NullValueCounts_Middle = sD_continuous_Middle.isnull().sum()
NullValueCols_Middle = NullValueCounts_Middle[NullValueCounts_Middle >= missingValueLimit].index
MiddleSchools = MiddleSchools.drop(NullValueCols_Middle, axis=1)

#Review dataset contents after empty field drops
print('\n')
print('*********After: Removing columns with >= missingThreshold % of missing values******')
print("************* Middle Schools *************")
MiddleSchools.info(verbose=False)
print ('\r\nColumns Deleted: ', len(NullValueCols_Middle))
print ('\r\nColumns: ', list(NullValueCols_Middle))

#Eliminate continuous columns with more than missingThreshold percentage of missing values
schoolDataRecordCt_Elementary = sD_continuous_Elementary.shape[0]
missingValueLimit_Elementary = schoolDataRecordCt_Elementary * missingThreshold
NullValueCounts_Elementary = sD_continuous_Elementary.isnull().sum()
NullValueCols_Elementary = NullValueCounts_Elementary[NullValueCounts_Elementary >= missingValueLimit].index
ElementarySchools = ElementarySchools.drop(NullValueCols_Elementary, axis=1)

#Review dataset contents after empty field drops
print('\n')
print('*********After: Removing columns with >= missingThreshold % of missing values******')
print("************* Elementary Schools *************")
ElementarySchools.info(verbose=False)
print ('\r\nColumns Deleted: ', len(NullValueCols_Elementary))
print ('\r\nColumns: ', list(NullValueCols_Elementary))




*********After: Removing columns with >= missingThreshold % of missing values******
************* All Schools *************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2443 entries, 2 to 2616
Columns: 157 entries, vphone_ad to RLPS
dtypes: float64(115), int64(12), object(30)
memory usage: 2.9+ MB

Columns Deleted:  86

Columns:  ['00_Size', '01_Size', '02_Size', '03_Size', '04_Size', '05_Size', '06_Size', '07_Size', '08_Size', '09_Size', '10_Size', '11_Size', '12_Size', '13_Size', 'Biology_Size', 'English II_Size', 'Math I_Size', 'Reading SPG Score', 'Math SPG Score', 'Reading Score', 'Math Score', 'Science Score', 'NC Math 1 Score', 'English II Score', 'Biology Score', 'ACT Score', 'ACT WorkKeys Score', 'Passing NC Math 3', '4-Year Cohort Graduation Rate Score', 'ACT_pTarget_PctMet', 'ACTWorkKeys_pTarget_PctMet', 'CurrentYearEOC_pTarget_PctMet', 'MathGr10_pTarget_PctMet', 'MathGr3-8_pTarget_PctMet', 'ReadingGr10_pTarget_PctMet', 'ReadingGr3-8_pTarget_PctMet', 'SciGr11_pTarget_

## Delete columns with more than 25 unique values before one hot encoding

In [23]:
#Delete categorical columns with > 25 unique values (Each unique value becomes a column during one-hot encoding)
oneHotUniqueValueCounts = schoolData[sD_nominal.columns].apply(lambda x: x.nunique())
oneHotUniqueValueCols = oneHotUniqueValueCounts[oneHotUniqueValueCounts >= uniqueThreshold].index
schoolData.drop(oneHotUniqueValueCols, axis=1, inplace=True) 

#Review dataset contents one hot high unique value drops
print('*********After: Removing columns with >= uniqueThreshold unique values***********')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(oneHotUniqueValueCols))
print ('\r\nColumns : ', list(oneHotUniqueValueCols))

#Delete categorical columns with > 25 unique values (Each unique value becomes a column during one-hot encoding)
oneHotUniqueValueCounts_High = HighSchools[sD_nominal_High.columns].apply(lambda x: x.nunique())
oneHotUniqueValueCols_High = oneHotUniqueValueCounts_High[oneHotUniqueValueCounts_High >= uniqueThreshold].index
HighSchools.drop(oneHotUniqueValueCols_High, axis=1, inplace=True) 

#Review dataset contents one hot high unique value drops
print('\n')
print('*********High Schools***********')
print('*********After: Removing columns with >= uniqueThreshold unique values***********')
HighSchools.info(verbose=False)
print ('\r\nColumns Deleted: ', len(oneHotUniqueValueCols_High))
print ('\r\nColumns : ', list(oneHotUniqueValueCols_High))

#Delete categorical columns with > 25 unique values (Each unique value becomes a column during one-hot encoding)
oneHotUniqueValueCounts_Middle = MiddleSchools[sD_nominal_Middle.columns].apply(lambda x: x.nunique())
oneHotUniqueValueCols_Middle = oneHotUniqueValueCounts_Middle[oneHotUniqueValueCounts_Middle >= uniqueThreshold].index
MiddleSchools.drop(oneHotUniqueValueCols, axis=1, inplace=True) 

#Review dataset contents one hot high unique value drops
print('\n')
print('*********Middle Schools***********')
print('*********After: Removing columns with >= uniqueThreshold unique values***********')
MiddleSchools.info(verbose=False)
print ('\r\nColumns Deleted: ', len(oneHotUniqueValueCols_Middle))
print ('\r\nColumns : ', list(oneHotUniqueValueCols_Middle))

#Delete categorical columns with > 25 unique values (Each unique value becomes a column during one-hot encoding)
oneHotUniqueValueCounts_Elementary = ElementarySchools[sD_nominal_Elementary.columns].apply(lambda x: x.nunique())
oneHotUniqueValueCols_Elementary = oneHotUniqueValueCounts_Elementary[oneHotUniqueValueCounts_Elementary >= uniqueThreshold].index
ElementarySchools.drop(oneHotUniqueValueCols_Elementary, axis=1, inplace=True) 

#Review dataset contents one hot high unique value drops
print('\n')
print('*********Elementary Schools***********')
print('*********After: Removing columns with >= uniqueThreshold unique values***********')
ElementarySchools.info(verbose=False)
print ('\r\nColumns Deleted: ', len(oneHotUniqueValueCols_Elementary))
print ('\r\nColumns : ', list(oneHotUniqueValueCols_Elementary))

*********After: Removing columns with >= uniqueThreshold unique values***********
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2443 entries, 2 to 2616
Columns: 143 entries, category_cd to RLPS
dtypes: float64(115), int64(12), object(16)
memory usage: 2.7+ MB

Columns Deleted:  14

Columns :  ['vphone_ad', 'street_ad', 'scity_ad', 'szip_ad', 'url_ad', 'grade_range_cd', 'cover_letter_ad', 'Lea_Name', 'url', 'District Name', 'School Name', 'grades_BYOD', 'grades_1_to_1_access', 'SRC_Grades_Devices_Sent_Home']


*********High Schools***********
*********After: Removing columns with >= uniqueThreshold unique values***********
<class 'pandas.core.frame.DataFrame'>
Int64Index: 470 entries, 2 to 2615
Columns: 233 entries, category_cd to RLPS
dtypes: float64(201), int64(12), object(20)
memory usage: 859.2+ KB

Columns Deleted:  10

Columns :  ['vphone_ad', 'street_ad', 'scity_ad', 'szip_ad', 'url_ad', 'cover_letter_ad', 'Lea_Name', 'url', 'District Name', 'School Name']


*********Middle S

## One hot encode categorical variables

In [24]:
#Isolate remaining categorical variables
begColumnCt = len(schoolData.columns)
sD_nominal = schoolData.loc[:, (schoolData.dtypes == object)]

#one hot encode categorical variables
schoolData = pd.get_dummies(data=schoolData, 
                       columns=sD_nominal, drop_first=True)

#Determine change in column count
endColumnCt = len(schoolData.columns)
columnsAdded = endColumnCt - begColumnCt

#Review dataset contents one hot high unique value drops
print('\n')
print('*********** All Schools ***********')
print ('Columns To One-Hot Encode: ', len(sD_nominal.columns), list(sD_nominal.columns))
print('\r\n*********After: Adding New Columns Via One-Hot Encoding*************************')
schoolData.info(verbose=False)
print ('\r\nNew Columns Created Via One-Hot Encoding: ', columnsAdded)


#Isolate remaining categorical variables
begColumnCt = len(HighSchools.columns)
sD_nominal_High = HighSchools.loc[:, (HighSchools.dtypes == object)]

#one hot encode categorical variables
HighSchools = pd.get_dummies(data=HighSchools, 
                       columns=sD_nominal_High, drop_first=True)

#Determine change in column count
endColumnCt = len(HighSchools.columns)
columnsAdded = endColumnCt - begColumnCt

#Review dataset contents one hot high unique value drops
print('\n')
print('*********** High Schools ***********')
print ('Columns To One-Hot Encode: ', len(sD_nominal_High.columns), list(sD_nominal_High.columns))
print('\r\n*********After: Adding New Columns Via One-Hot Encoding*************************')
HighSchools.info(verbose=False)
print ('\r\nNew Columns Created Via One-Hot Encoding: ', columnsAdded)

#Isolate remaining categorical variables
begColumnCt = len(MiddleSchools.columns)
sD_nominal_Middle = MiddleSchools.loc[:, (MiddleSchools.dtypes == object)]

#one hot encode categorical variables
MiddleSchools = pd.get_dummies(data=MiddleSchools, 
                       columns=sD_nominal_Middle, drop_first=True)

#Determine change in column count
endColumnCt = len(MiddleSchools.columns)
columnsAdded = endColumnCt - begColumnCt

#Review dataset contents one hot high unique value drops
print('\n')
print('*********** Middle Schools ***********')
print ('Columns To One-Hot Encode: ', len(sD_nominal_Middle.columns), list(sD_nominal_Middle.columns))
print('\r\n*********After: Adding New Columns Via One-Hot Encoding*************************')
MiddleSchools.info(verbose=False)
print ('\r\nNew Columns Created Via One-Hot Encoding: ', columnsAdded)

#Isolate remaining categorical variables
begColumnCt = len(ElementarySchools.columns)
sD_nominal_Elementary = ElementarySchools.loc[:, (ElementarySchools.dtypes == object)]

#one hot encode categorical variables
ElementarySchools = pd.get_dummies(data=ElementarySchools, 
                       columns=sD_nominal_Elementary, drop_first=True)

#Determine change in column count
endColumnCt = len(ElementarySchools.columns)
columnsAdded = endColumnCt - begColumnCt

#Review dataset contents one hot high unique value drops
print('\n')
print('*********** Elementary Schools ***********')
print ('Columns To One-Hot Encode: ', len(sD_nominal_Elementary.columns), list(sD_nominal_Elementary.columns))
print('\r\n*********After: Adding New Columns Via One-Hot Encoding*************************')
ElementarySchools.info(verbose=False)
print ('\r\nNew Columns Created Via One-Hot Encoding: ', columnsAdded)




*********** All Schools ***********
Columns To One-Hot Encode:  16 ['category_cd', 'calendar_type_txt', 'sna_pgm_type_cd', 'school_type_txt', 'calendar_only_txt', 'esea_status', 'SBE Region', 'SPG Grade', 'Reading SPG Grade', 'Math SPG Grade', 'EVAAS Growth Status', 'State Gap Compared', 'Category_Cd', 'Byod', '_1_to_1_access', 'SRC_devices_sent_home']

*********After: Adding New Columns Via One-Hot Encoding*************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2443 entries, 2 to 2616
Columns: 186 entries, title1_type_cd to SRC_devices_sent_home_Yes
dtypes: float64(115), int64(12), uint8(59)
memory usage: 2.5 MB

New Columns Created Via One-Hot Encoding:  43


*********** High Schools ***********
Columns To One-Hot Encode:  20 ['category_cd', 'grade_range_cd', 'calendar_type_txt', 'sna_pgm_type_cd', 'school_type_txt', 'calendar_only_txt', 'esea_status', 'SBE Region', 'SPG Grade', 'Reading SPG Grade', 'Math SPG Grade', 'EVAAS Growth Status', 'State Gap Compared', 

### Impute remaining missing values to zero

In [25]:
#Print out all the missing value rows
pd.set_option('display.max_rows', 1000)

print('\r\n*********The Remaining Missing Values Below will be set to Zero!*************************')

#Check for Missing values 
missing_values = schoolData.isnull().sum().reset_index()
missing_values.columns = ['Variable Name', 'Number Missing Values']
missing_values = missing_values[missing_values['Number Missing Values'] > 0] 
print(missing_values)

#Check for Missing values 
missing_values_High = HighSchools.isnull().sum().reset_index()
missing_values_High.columns = ['Variable Name', 'Number Missing Values']
missing_values_High = missing_values_High[missing_values_High['Number Missing Values'] > 0] 


#Check for Missing values 
missing_values_Middle = MiddleSchools.isnull().sum().reset_index()
missing_values_Middle.columns = ['Variable Name', 'Number Missing Values']
missing_values_Middle = missing_values_Middle[missing_values_Middle['Number Missing Values'] > 0] 


#Check for Missing values 
missing_values_Elementary = ElementarySchools.isnull().sum().reset_index()
missing_values_Elementary.columns = ['Variable Name', 'Number Missing Values']
missing_values_Elementary = missing_values_Elementary[missing_values_Elementary['Number Missing Values'] > 0] 



*********The Remaining Missing Values Below will be set to Zero!*************************
                                        Variable Name  Number Missing Values
17                                          SPG Score                    130
18                                 EVAAS Growth Score                    142
19                          Overall Achievement Score                    130
20                        TotalTargets_pTarget_PctMet                     68
21                              lea_sat_avg_score_num                      6
22                          lea_sat_participation_pct                      6
23                           lea_ap_participation_pct                    263
24                              lea_ap_pct_3_or_above                     64
30                             stud_internet_comp_num                      8
37                                  digital_media_pct                    142
38                           avg_age_media_collection         

In [26]:
#Replace all remaining NaN with 0
schoolData = schoolData.fillna(0)
HighSchools = HighSchools.fillna(0)
MiddleSchools = MiddleSchools.fillna(0)
ElementarySchools = ElementarySchools.fillna(0)

##  Last Drop

In [27]:
redundantFields = schoolData.filter(regex=('_spg_ind|sna_pgm_type_cd|Years_Exp_Pct_Tch|Category_Cd_|school_type_txt_')).columns
keepFields = [col for col in schoolData.columns if col not in redundantFields]
schoolData = schoolData[keepFields]
print(schoolData.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2443 entries, 2 to 2616
Columns: 164 entries, title1_type_cd to SRC_devices_sent_home_Yes
dtypes: float64(112), int64(10), uint8(42)
memory usage: 2.4 MB
None


In [28]:
#Restore the unit_code before saving
schoolData['unit_code'] = unit_code
HighSchools['unit_code'] = unit_code
MiddleSchools['unit_code'] = unit_code
ElementarySchools['unit_code'] = unit_code
#Save the final dataset to a .csv file
schoolData.to_csv(outputDir + 'PublicSchools' + str(schoolYear) + '_LPS_Processed' + '.csv', sep=',', index=False)
HighSchools.to_csv(outputDir + 'PublicHighSchools' + str(schoolYear) + '_LPS_Processed' + '.csv', sep=',', index=False)
MiddleSchools.to_csv(outputDir + 'PublicMiddleSchools' + str(schoolYear) + '_LPS_Processed' + '.csv', sep=',', index=False)
ElementarySchools.to_csv(outputDir + 'PublicElementarySchools' + str(schoolYear) + '_LPS_Processed' +'.csv', sep=',', index=False)


In [29]:
print('*********FINAL DATASET DETAILS*********************************************************\r\n')
schoolData.info(verbose=True)
HighSchools.info(verbose=True)
MiddleSchools.info(verbose=True)
ElementarySchools.info(verbose=True)

*********FINAL DATASET DETAILS*********************************************************

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2443 entries, 2 to 2616
Data columns (total 165 columns):
title1_type_cd                                                  int64
clp_ind                                                         int64
focus_clp_ind                                                   int64
summer_program_ind                                              int64
student_num                                                     float64
lea_avg_student_num                                             float64
Grad_project_status                                             int64
lea_total_expense_num                                           float64
lea_salary_expense_pct                                          float64
lea_services_expense_pct                                        float64
lea_supplies_expense_pct                                        float64
lea_instruct_equip_ex

In [30]:
schoolData.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2443 entries, 2 to 2616
Data columns (total 165 columns):
title1_type_cd                                                  int64
clp_ind                                                         int64
focus_clp_ind                                                   int64
summer_program_ind                                              int64
student_num                                                     float64
lea_avg_student_num                                             float64
Grad_project_status                                             int64
lea_total_expense_num                                           float64
lea_salary_expense_pct                                          float64
lea_services_expense_pct                                        float64
lea_supplies_expense_pct                                        float64
lea_instruct_equip_exp_pct                                      float64
lea_federal_perpupil_num              